# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
### Custon training walkthrough
Categorizing Iris flowers by species by using Tensorflow's eager execution.

This guide uses these high-level TensorFlow concepts:

* Enable an [eager execution](https://www.tensorflow.org/guide/eager?hl=ko) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets?hl=ko)
* Build models and layers with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/)  
  
  
* Reference
    + https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough?hl=ko

### Setup

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()

### Create a model

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape = (4,)))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3))

In [3]:
def loss_fn(model, features, label):
    score = model(features)
    return tf.losses.sparse_softmax_cross_entropy(labels = label, logits = score)

In [4]:
print(model.trainable_variables)

[<tf.Variable 'dense/kernel:0' shape=(4, 10) dtype=float32, numpy=
array([[ 0.23273689,  0.59694576,  0.53505063, -0.41671774,  0.5939325 ,
         0.5985272 ,  0.16594094,  0.22990024,  0.24404621, -0.57850736],
       [ 0.50012505,  0.28323126,  0.51059794, -0.34209886,  0.07642943,
         0.42025232, -0.3534193 , -0.25960475,  0.3988968 ,  0.13867897],
       [ 0.37427866,  0.11757517, -0.5074648 ,  0.27821964,  0.64113617,
         0.54645467, -0.5339945 , -0.32484716,  0.5371665 ,  0.52713263],
       [ 0.64209175,  0.27604747, -0.64053625, -0.14172429,  0.12444371,
         0.14510643,  0.14198619,  0.34790993, -0.41857135, -0.6528216 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
array([[-0.44230747,  0.31619924,  0.426068  , -0.10556999, -0.32908076,
        -0.10463578,  0.43683833, -0.28880256,  0.384

### Import and parse the training dataset

In [5]:
os.listdir('../data/lecture04/')

['iris_training.csv', 'iris_test.csv']

In [6]:
# define parsing function
def parse_single_example(record):
    decoded = tf.decode_csv(record, [[.0],[.0],[.0],[.0],[]])
    features = decoded[:4]
    label = tf.cast(x = decoded[4], dtype = tf.int32)
    return features, label

In [7]:
epochs = 10
batch_size = 8
learning_rate = .03
tr_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_training.csv')
tr_dataset = tr_dataset.map(parse_single_example)
tr_dataset = tr_dataset.shuffle(200).batch(batch_size = batch_size)
opt = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
global_step = tf.Variable(0.)

In [8]:
for epoch in range(epochs):
    avg_loss = 0
    tr_step = 0
    
    for mb_x, mb_y in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, mb_x, mb_y)
        grads = tape.gradient(tr_loss, model.variables)
        opt.apply_gradients(zip(grads, model.variables), global_step = global_step)
        
        avg_loss += tr_loss
        tr_step += 1
    else:
        avg_loss /= tr_step
    
    print('epoch : {:3}, ce_loss : {:.3f}'.format(epoch + 1, avg_loss))

epoch :   1, ce_loss : 0.997
epoch :   2, ce_loss : 0.758
epoch :   3, ce_loss : 0.621
epoch :   4, ce_loss : 0.537
epoch :   5, ce_loss : 0.493
epoch :   6, ce_loss : 0.457
epoch :   7, ce_loss : 0.427
epoch :   8, ce_loss : 0.394
epoch :   9, ce_loss : 0.368
epoch :  10, ce_loss : 0.326


### Evaluate the model on the test dataset

In [9]:
tst_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_test.csv')
tst_dataset = tst_dataset.map(parse_single_example)
tst_dataset = tst_dataset.batch(batch_size = 30)

In [10]:
tst_x, tst_y = next(iter(tst_dataset))
tst_yhat = tf.argmax(model(tst_x), axis = -1, output_type = tf.int32) 

In [11]:
print('test accuracy : {:.2%}'.format(np.mean(tf.equal(tst_y, tst_yhat))))

test accuracy : 93.33%
